### 1 - Import test data

In [1]:
!pip3 install yfinance
!pip3 install pandas


In [8]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X",  period = "7d",
 interval='15m',  auto_adjust = True,  prepost = False,)
dataF.iloc[:,:]
#dataF.Open.iloc

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Datetime,,,,,
2023-10-26 00:00:00,1.056859,1.056859,1.056747,1.056747,0
2023-10-26 00:15:00,1.056747,1.056747,1.056301,1.056524,0
2023-10-26 00:30:00,1.056524,1.056747,1.056524,1.056636,0
2023-10-26 00:45:00,1.056636,1.056971,1.056636,1.056859,0
2023-10-26 01:00:00,1.056971,1.057194,1.056859,1.056971,0
...,...,...,...,...,...
2023-11-03 20:15:00,1.073076,1.073307,1.072961,1.073307,0
2023-11-03 20:30:00,1.073422,1.073537,1.073422,1.073422,0
2023-11-03 20:45:00,1.073422,1.073422,1.073076,1.073307,0


### 2 - Define your signal function

In [9]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
#signal_generator(data)
dataF["signal"] = signal

In [11]:
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    598
1     32
2     26
Name: count, dtype: int64

### 3 - Connect to the market and execute trades

In [12]:
!pip3 install apscheduler
!pip3 install oandapyV20
!pip3 install oanda_candles

In [13]:
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails

In [19]:
# from config import access_token, accountID
def get_candles(n):
    access_token='12345678900987654321-abc34135acde13f13530'#you need token here generated from OANDA account
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

candles = get_candles(3)
for candle in candles:
    print(float(str(candle.bid.o))>1)


HTTPError: 401 Client Error: Unauthorized for url: https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles?alignmentTimezone=Etc%2FGMT%2B1&dailyAlignment=23&granularity=M15&price=BAM&weeklyAlignment=Sunday&count=3

In [16]:
def trading_job():
    candles = get_candles(3)
    dfstream = pd.DataFrame(columns=['Open','Close','High','Low'])
    
    i=0
    for candle in candles:
        dfstream.loc[i, ['Open']] = float(str(candle.bid.o))
        dfstream.loc[i, ['Close']] = float(str(candle.bid.c))
        dfstream.loc[i, ['High']] = float(str(candle.bid.h))
        dfstream.loc[i, ['Low']] = float(str(candle.bid.l))
        i=i+1

    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])#
    
    # EXECUTING ORDERS
    #accountID = "XXXXXXX" #your account ID here
    client = API(access_token)
         
    SLTPRatio = 2.
    previous_candleR = abs(dfstream['High'].iloc[-2]-dfstream['Low'].iloc[-2])
    
    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio
    
    print(dfstream.iloc[:-1,:])
    print(TPBuy, "  ", SLBuy, "  ", TPSell, "  ", SLSell)
    signal = 2
    #Sell
    if signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-1000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)
    #Buy
    elif signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=1000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### 4 - Executing orders automatically with a scheduler

In [17]:
trading_job()

#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,16,31,46', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
#scheduler.start()

NameError: name 'get_candles' is not defined